# Project: ICD-AIS conversion using Deep Learning

This converts ICD codes to AIS using the AAAM ICD-AIS map

## Setup

In [21]:
import numpy as np
import pandas as pd
import sqlite3
import random
import math
import modules.helper_functions_100 as hlp

## Parameters


In [22]:
icd_obs_file ="../Data/test_icd_pre_I9_A05.csv" # Looks to be a list of ICD codes associated with patient injuries
ais_map_codes_file = "../Data/AIS08_mapped_codes.csv" # Maps AIS08 to AIS98
aaam_map_file = "../Data/AAAM ICD-ISS Map v2.0 (00082).xlsx" # Maps ICD to ISS; so you have in here ICD code, description, highest AIS, ISS body region, and AIS chapter (which correlate to body region)
aaam_map_sheet = "icd9Map" # This is not utilixed elsewere but in loading the data, so probably can be deleted

## Load data

In [23]:
# load ICD codes
icd_codes = pd.read_csv(icd_obs_file, header=None, names=["ICD9CODE"]) # Load the csv data of patient injuries into an object
icd_codes.head(5)

,ICD9CODE
0,A79 E880.9 P87.03 P87.44 D850.0
1,A55 E884.9 P-2 D805.2 D805.4 D807.03
2,A70 E885.9 P79.35 P87.03 D820.21
3,A56 E881.0 P88.01 P88.38 D805.4 D823.01 D825.20
4,A40 E986 P54.11 D879.4


In [24]:
# load ICD-AIS map
aaam_map = pd.read_excel(aaam_map_file, sheet_name="icd9Map", header=0, converters={'CODE':str}) # Load data that maps ICD to ISS into an object

In [25]:
aaam_map.head(5) # Print the first 5 entries in the ICD to ISS map (run to populate below)

,CODE,DESCRIPTION,HIGHEST AIS SEVERITY,ISS BODY REGION,AIS CHAPTER
0,800,Fracture Of Vault Of Skull,2,1,1
1,800.0,Closed Fracture Of Vault Of Skull Without Ment...,2,1,1
2,800.00,Closed fracture of vault of skull without ment...,2,1,1
3,800.01,Closed fracture of vault of skull without ment...,2,1,1
4,800.02,Closed fracture of vault of skull without ment...,2,1,1


## Create AIS code from Information

Since AIS codes are typically [Chapter]XXXXX.[Severity], we will transform this to [Chapter][Region]0000.[Severity].  These will not overlap any valid AIS08 codes.

In [26]:
# So below we add a new column to the AAAM map from above called AIS_CODE. This is a code we are building from scratch to map
# So the first part of the dev code is the chapter
# Second is the region of the body?
# The last, after the period is severity
# We calculate this dev code below with some math
aaam_map['AIS_CODE'] = aaam_map['AIS CHAPTER'] * 100_000 + aaam_map['ISS BODY REGION'] * 10_000 + aaam_map['HIGHEST AIS SEVERITY'] * 0.1

In [27]:
aaam_map.head(5)

,CODE,DESCRIPTION,HIGHEST AIS SEVERITY,ISS BODY REGION,AIS CHAPTER,AIS_CODE
0,800,Fracture Of Vault Of Skull,2,1,1,110000.2
1,800.0,Closed Fracture Of Vault Of Skull Without Ment...,2,1,1,110000.2
2,800.00,Closed fracture of vault of skull without ment...,2,1,1,110000.2
3,800.01,Closed fracture of vault of skull without ment...,2,1,1,110000.2
4,800.02,Closed fracture of vault of skull without ment...,2,1,1,110000.2


## Convert ICD codes with tidy format

In [28]:
icd_codes = hlp.tidy_icd_code(icd_codes)

In [29]:
icd_codes.head(5)

,key,icd9_code
0,0,A79
1,0,E880.9
2,0,P87.03
3,0,P87.44
4,0,D850.0


## Trim to only diagnosis codes

In [30]:
icd_codes = hlp.trim_codes(icd_codes)
icd_codes.head(5)

,key,icd9_code
0,0,850.0
1,1,805.2
2,1,805.4
3,1,807.03
4,2,820.21


In [31]:
len(icd_codes)

351765

## Map ICD9 to AIS 08

In [32]:
# Merge map onto ICD codes
icd_codes = hlp.ICD9_AIS08_map(aaam_map, icd_codes, 'AAAM')
icd_codes.head(5)

,key,icd9_code,CODE,AIS_CODE
0,0,850.0,850.0,110000.1
3,1,807.03,807.03,420000.3
1,1,805.2,805.2,620000.2
2,1,805.4,805.4,630000.2
4,2,820.21,820.21,840000.3


## Evaluate map

In [33]:
# number of codes that do not map
print("Unmatched", sum(icd_codes.AIS_CODE==-1))
print("% Unmatched", sum(icd_codes.AIS_CODE==-1)/len(icd_codes)*100)

Unmatched 9972
% Unmatched 2.834847128054241


In [34]:
# number of unspecified codes
print("Unspecified", sum(icd_codes.AIS_CODE==0))
print("% Unspecified", sum(icd_codes.AIS_CODE==0)/len(icd_codes)*100)

Unspecified 1711
% Unspecified 0.48640427558170934


In [35]:
# number of codes
len(icd_codes)

351765

In [36]:
# number of patients
len(icd_codes.key.unique())

99716

## Convert to codes list

In [37]:
icd_codes = hlp.convert_codes_to_lists(icd_codes)
icd_codes.head(5)

,AIS_CODE
0,[110000.1]
1,"[420000.3, 620000.2, 630000.2]"
2,[840000.3]
3,"[630000.2, 840000.2, 840000.2]"
4,[560000.1]


## Store results

In [38]:
icd_codes.to_csv("../Results/test_ais_pred_aaam_map.csv", index=False, header=False)